In [1]:
### Installation

%%capture
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

In [2]:
### Unsloth

from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "microsoft/Phi-3.5-mini-instruct", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.12: Fast Mistral patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA GeForce RTX 3060. Max memory: 12.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.6. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
### LORA

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.12 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


In [4]:
### Data Prep

from unsloth.chat_templates import get_chat_template
from datasets import Dataset
import pandas as pd

# Получаем базовый шаблон chatml
tokenizer = get_chat_template(
    tokenizer,
    chat_template="chatml",
    map_eos_token=True,
)

# Настраиваем токены
tokenizer.add_special_tokens({
    "additional_special_tokens": [
        "<|system|>",
        "<|user|>",
        "<|assistant|>",
        "<|end|>",
    ],
})

# Устанавливаем шаблон Phi
tokenizer.chat_template = """{% for message in messages %}{% if message['role'] == 'system' and message['content'] %}{{'<|system|>\n' + message['content'] + '<|end|>\n'}}{% elif message['role'] == 'user' %}{{'<|user|>\n' + message['content'] + '<|end|>\n'}}{% elif message['role'] == 'assistant' %}{{'<|assistant|>\n' + message['content'] + '<|end|>\n'}}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>\n' }}{% else %}{{ eos_token }}{% endif %}"""

def formatting_prompts_func(examples):
    convos = []
    for system, user, assistant in zip(examples["system"], examples["user"], examples["assistant"]):
        messages = [
            {"role": "system", "content": system},
            {"role": "user", "content": user},
            {"role": "assistant", "content": assistant}
        ]
        formatted_text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )
        convos.append(formatted_text)
        
        # Выводим первый пример для отладки
        if len(convos) == 1:
            print("\nПример форматированного текста:")
            print("-" * 50)
            print(formatted_text)
            print("-" * 50)
            
    return {"text": convos}

# Загрузка данных
df = pd.read_parquet("dataset/merged_dataset_hb_cl.parquet")[["system", "user", "assistant"]]
dataset = Dataset.from_pandas(df)
dataset = dataset.map(formatting_prompts_func, batched=True)

Unsloth: Will map <|im_end|> to EOS = </s>.


Map:   0%|          | 0/20538 [00:00<?, ? examples/s]


Пример форматированного текста:
--------------------------------------------------
<s><|start_header_id|>user<|end_header_id|>

Можете ли вы описать ощущение падения в реальности?</s><|start_header_id|>assistant<|end_header_id|>

Ой, да ладно тебе. Конечно же, я могу описать это ощущение, и даже очень детально. Представь себе: ты стоишь на каком-нибудь высоком месте — скажем, на крыше дома или на обрыве у моря. Вдруг под ногами начинает проваливаться земля, и ты чувствуешь, как теряешь опору. Сердце сразу же замирает в груди, а потом начинает биться так сильно, что кажется, вот-вот выпрыгнет наружу. Ты пытаешься схватить воздух руками, но он скользит сквозь пальцы, не давая опоры. В ушах звенит, и ты словно слышишь свой собственный крик — громкий, полный ужаса. Весь мир вокруг начинает замедляться: каждый миллиметр падения кажется вечностью. Ты чувствуешь, как воздух обдувает лицо, а волосы разлетаются в стороны. Тело напрягается до предела — все мышцы сжимаются, готовые к удару о зем

In [5]:
### Train the model

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        learning_rate = 1e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs_phi_3.5",
        report_to = "none", # Use this for WandB etc
    ),
)

Applying chat template to train dataset (num_proc=2):   0%|          | 0/20538 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/20538 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/20538 [00:00<?, ? examples/s]

In [6]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3060. Max memory = 12.0 GB.
8.004 GB of memory reserved.


In [ ]:
### Train Start

trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 20,538 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 2,567
 "-____-"     Number of trainable parameters = 28,508,160


Step,Training Loss
1,2.978800
2,3.227800
3,2.756700
4,2.753600
5,2.981000
6,2.579700
7,2.541300
8,2.137600
9,2.003600
10,1.590000


In [ ]:
### Saving, loading finetuned models

model.save_pretrained("lora_model_phi_3.5")  # Local saving
tokenizer.save_pretrained("lora_model_phi_3.5")